In [1]:
import pandas as pd

X_train_valid = pd.read_csv('train.csv', index_col='Id')
X_test = pd.read_csv('test.csv', index_col='Id')

In [2]:
X_train_valid

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500


In [3]:
X_train_valid.dropna(subset=['SalePrice'], axis=0, inplace=True)

y_train_valid = X_train_valid.SalePrice
X_train_valid.drop(['SalePrice'], axis=1, inplace=True)

In [4]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X_train_valid, y_train_valid, test_size=0.2, random_state=0)

In [5]:
object_cols = [col for col in X_train.columns if 
               X_train[col].dtype == 'object']

good_label_cols = [col for col in object_cols if 
                   set(X_train[col]) == set(X_valid[col])]
bad_label_cols = list(set(object_cols)-set(good_label_cols))

print(good_label_cols)
print(bad_label_cols)

['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'LotConfig', 'BldgType', 'HouseStyle', 'MasVnrType', 'ExterQual', 'BsmtQual', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'CentralAir', 'KitchenQual', 'FireplaceQu', 'GarageFinish', 'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'SaleCondition']
['Condition1', 'Exterior2nd', 'RoofMatl', 'Utilities', 'MiscFeature', 'GarageType', 'Condition2', 'Heating', 'Functional', 'Electrical', 'Foundation', 'Exterior1st', 'HeatingQC', 'BsmtCond', 'RoofStyle', 'SaleType', 'Neighborhood', 'GarageQual', 'LandSlope', 'ExterCond']


In [6]:
X_train.drop(bad_label_cols, axis=1, inplace=True)
X_valid.drop(bad_label_cols, axis=1, inplace=True)
X_train_valid.drop(bad_label_cols, axis=1, inplace=True)
X_test.drop(bad_label_cols, axis=1, inplace=True)

In [7]:
low_cardinality_categorical_cols = [col for col in X_train.columns if 
                                    X_train[col].nunique() < 10 and 
                                    X_train[col].dtype == 'object']

numerical_cols = [col for col in X_train.columns if 
                  X_train[col].dtype in ['int64', 'float64']]

kept_cols = low_cardinality_categorical_cols + numerical_cols

X_train = X_train[kept_cols].copy()
X_valid = X_valid[kept_cols].copy()
X_train_valid = X_train_valid[kept_cols].copy()

for col in X_train.columns:
    if X_train[col].isnull().sum() != 0:
        X_train[col+'_was_missing'] = X_train[col].isnull().astype('category').cat.codes
        X_valid[col+'_was_missing'] = X_valid[col].isnull().astype('category').cat.codes
        X_train_valid[col+'_was_missing'] = X_train_valid[col].isnull().astype('category').cat.codes
        X_test[col+'_was_missing'] = X_test[col].isnull().astype('category').cat.codes

In [8]:
X_train_valid

,MSZoning,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Condition1,Condition2,...,GarageType_was_missing,GarageFinish_was_missing,GarageQual_was_missing,GarageCond_was_missing,PoolQC_was_missing,Fence_was_missing,MiscFeature_was_missing,LotFrontage_was_missing,MasVnrArea_was_missing,GarageYrBlt_was_missing
Id,,,,,,,,,,,,,,,,,,,,,
1,RL,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Norm,Norm,...,0,0,0,0,1,1,1,0,0,0
2,RL,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Feedr,Norm,...,0,0,0,0,1,1,1,0,0,0
3,RL,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Norm,Norm,...,0,0,0,0,1,1,1,0,0,0
4,RL,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Norm,Norm,...,0,0,0,0,1,1,1,0,0,0
5,RL,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,Norm,Norm,...,0,0,0,0,1,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1456,RL,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Norm,Norm,...,0,0,0,0,1,1,1,0,0,0
1457,RL,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Norm,Norm,...,0,0,0,0,1,0,1,0,0,0
1458,RL,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Norm,Norm,...,0,0,0,0,1,0,0,0,0,0


In [9]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV

numerical_transformer = SimpleImputer()

categorical_transformer = Pipeline(steps=[
    ('imp', SimpleImputer(strategy='most_frequent')), 
    ('ohe', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
     ])

preprocessor = ColumnTransformer(transformers=[
        ('num', numerical_transformer, numerical_cols), 
        ('cat', categorical_transformer, low_cardinality_categorical_cols)
    ])

model = RandomForestRegressor(random_state=0)

clf = Pipeline(steps=[
    ('preprocessor', preprocessor), 
    ('model', model)
])

param_grid = {
    'preprocessor__num__strategy': ['mean', 'median'], 
    'model__n_estimators': range(50, 550, 50)
}

search = GridSearchCV(clf, param_grid, scoring='neg_mean_absolute_error', n_jobs=-1, verbose=3)
search.fit(X_train_valid, y_train_valid)
print("Best parameter (CV score=%0.3f):" % search.best_score_)
print(search.best_params_)

Fitting 5 folds for each of 20 candidates, totalling 100 fits
Best parameter (CV score=-17532.798):
{'model__n_estimators': 200, 'preprocessor__num__strategy': 'mean'}


In [10]:
y_pred_rfr = search.best_estimator_.predict(X_test)

In [11]:
model = GradientBoostingRegressor(random_state=0)

clf = Pipeline(steps=[
    ('preprocessor', preprocessor), 
    ('model', model)
])

param_grid = {
    'preprocessor__num__strategy': ['mean', 'median'], 
    'model__n_estimators': range(50, 550, 50)
}

search = GridSearchCV(clf, param_grid, scoring='neg_mean_absolute_error', n_jobs=-1, verbose=3)
search.fit(X_train_valid, y_train_valid)
print("Best parameter (CV score=%0.3f):" % search.best_score_)
print(search.best_params_)

Fitting 5 folds for each of 20 candidates, totalling 100 fits
Best parameter (CV score=-15755.374):
{'model__n_estimators': 500, 'preprocessor__num__strategy': 'median'}


In [12]:
y_pred_gbr = search.best_estimator_.predict(X_test)

In [13]:
y_pred = (y_pred_rfr + y_pred_gbr) / 2

In [14]:
output = pd.DataFrame({'Id': X_test.index, 'SalePrice': list(y_pred)})
output

,Id,SalePrice
0,1461,121674.739865
1,1462,162803.165098
2,1463,180123.091141
3,1464,186404.489359
4,1465,194233.391960
...,...,...
1454,2915,80839.760697
1455,2916,77936.312796
1456,2917,154637.468442
1457,2918,116762.489898


In [15]:
output.to_csv('submission.csv', index=False)